In [24]:
import numpy as np  
import pandas as pd 
import re           
from keras.preprocessing.text import Tokenizer 
from keras_preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings

In [109]:
animeDf = pd.read_csv('./anime_with_synopsis.csv')
animeDf = animeDf.drop("Genres",axis=1)
animeDf = animeDf.rename(columns={"sypnopsis": "Synopsis"},)
animeDf.head()

,MAL_ID,Name,Score,Synopsis
0,1,Cowboy Bebop,8.78,"In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"other day, another bounty—such is the life of ..."
2,6,Trigun,8.24,"Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,7.27,ches are individuals with special powers like ...
4,8,Bouken Ou Beet,6.98,It is the dark century and the people are suff...


In [110]:
honorifics = set(['Mr.', 'Mrs.', 'Ms.', 'Dr.', 'Prof.', 'Rev.', 'Capt.', 'Lt.-Col.', 
'Col.', 'Lt.-Cmdr.', 'The Hon.', 'Cmdr.', 'Flt. Lt.', 'Brgdr.', 'Wng. Cmdr.', 
'Group Capt.' ,'Rt.', 'Maj.-Gen.', 'Rear Admrl.', 'Esq.', 'Mx', 'Adv', 'Jr.'] )
stopwords = '''a
about
above
after
again
against
all
am
an
and
any
are
aren't
as
at
be
because
been
before
being
below
between
both
but
by
can't
cannot
could
couldn't
did
didn't
do
does
doesn't
doing
don't
down
during
each
few
for
from
further
had
hadn't
has
hasn't
have
haven't
having
he
he'd
he'll
he's
her
here
here's
hers
herself
him
himself
his
how
how's
i
i'd
i'll
i'm
i've
if
in
into
is
isn't
it
it's
its
itself
let's
me
more
most
mustn't
my
myself
no
nor
not
of
off
on
once
only
or
other
ought
our
ours	ourselves
out
over
own
same
shan't
she
she'd
she'll
she's
should
shouldn't
so
some
such
than
that
that's
the
their
theirs
them
themselves
then
there
there's
these
they
they'd
they'll
they're
they've
this
those
through
to
too
under
until
up
very
was
wasn't
we
we'd
we'll
we're
we've
were
weren't
what
what's
when
when's
where
where's
which
while
who
who's
whom
why
why's
with
won't
would
wouldn't
you
you'd
you'll
you're
you've
your
yours
yourself
yourselves'''.split('\n')

In [137]:
import re
import string
def removeStopWords(sent):
    try:
        text = [word for word in re.split("\W+",sent) if word and word.lower() not in stopwords]  # filter out empty words
        return ' '.join(text)
    except:
        print(text)

def splitAndStrip(sent):
    if not pd.isnull((sent)):
        final = []
        split_l = sent.split(' ')

        for word in split_l:
            word = re.sub('\'','',word)
            word = re.sub(',',' ',word)
            if '.' in word and word not in honorifics:
                final.append(word + '\n')
                continue
            final.append(word)
        final = ' '.join(final).rstrip()
        return final.split('\n')
    return ""

def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')
def processData(df):
    x = df.Synopsis.map(lambda x: splitAndStrip(x))
    return x
def weighWords(para):
    counts = {}
    for x in para:
        for y in x:
            for k in y:
                if k == '':
                    continue
                k = k.lower()
                if counts.get(k) is None:
                    counts[k] = 1
                else:
                    total = counts.get(k)
                    total = total+1
                    counts[k] = total
    return counts

In [142]:
x = processData(animeDf)
x = x.map(lambda x: [removeStopWords(g) for g in [y for y in x]])
# t = weighWords(x)
# t = pd.Series(t)

['Vash Stampede man 60 000 000 000 bounty head', 'reason hes merciless villain lays waste oppose flattens entire cities fun garnering title Humanoid Typhoon', 'leaves trail death destruction wherever goes anyone can count dead much make eye contact rumors say', 'actuality Vash huge softie claims never taken life avoids violence costs', 'crazy doughnut obsession buffoonish attitude tow Vash traverses wasteland planet Gunsmoke followed two insurance agents Meryl Stryfe Milly Thompson attempt minimize impact public', 'soon misadventures evolve life death situations group legendary assassins summoned bring suffering trio', 'Vashs agonizing past will unraveled morality principles pushed breaking point']


In [8]:
t = t.where([len(x) > 3 for x in t.keys()]).dropna()
maxFreq = t.max()
weights = t.map(lambda x: x/maxFreq)
sentenceDict = []
def sumWeights(para):
    for sent in para:
        sentenceWeight = []
        for word in sent:
            w = weights.get(word.lower())
            if w is not None:
                sentenceWeight.append(float(w)/len(sent))
        sentenceDict.append(round(sum(sentenceWeight), 4))
    return sentenceDict

In [21]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
text = flatten_list(x[0])
vectorizer = CountVectorizer()
vectorizer.fit(text)
vectorizer2 = TfidfVectorizer()
vectorizer2.fit(text)
hashv = HashingVectorizer(n_features=20)
v = hashv.transform(text)
v.toarray()
print(text)

['year', '2071', 'humanity', 'colonized', 'several', 'planets', 'moons', 'solar', 'system', 'leaving', 'now', 'uninhabitable', 'surface', 'planet', 'Earth', 'behind', 'Inter', 'Solar', 'System', 'Police', 'attempts', 'keep', 'peace', 'galaxy', 'aided', 'part', 'outlaw', 'bounty', 'hunters', 'referred', 'Cowboys', 'ragtag', 'team', 'aboard', 'spaceship', 'Bebop', 'two', 'individuals', 'Mellow', 'carefree', 'Spike', 'Spiegel', 'balanced', 'boisterous', 'pragmatic', 'partner', 'Jet', 'Black', 'pair', 'makes', 'living', 'chasing', 'bounties', 'collecting', 'rewards', 'Thrown', 'course', 'addition', 'new', 'members', 'meet', 'travels', 'Ein', 'genetically', 'engineered', 'highly', 'intelligent', 'Welsh', 'Corgi', 'femme', 'fatale', 'Faye', 'Valentine', 'enigmatic', 'trickster', 'memory', 'loss', 'strange', 'computer', 'whiz', 'kid', 'Edward', 'Wong', 'crew', 'embarks', 'thrilling', 'adventures', 'unravel', 'member', 's', 'dark', 'mysterious', 'past', 'little', 'little', 'Well', 'balanced', 